In [1]:
from tmdbv3api import TMDb
from tmdbv3api import Movie
from tmdbv3api import TV
from tmdbv3api import Genre, Search

tmdb = TMDb()
tmdb.api_key = '6727d8b68d506e9160ce54956c29d855'
tmdb.language = 'en'
tmdb.debug = True

movie = Movie()
tv = TV()
genre = Genre()

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import json as json
import pickle

In [3]:
stc = pd.read_csv('/Users/dinaabed/Desktop/Final_Dataset.txt', delimiter = ",")
stc = stc.drop('Unnamed: 0', axis=1)
df = stc.copy() # the data is heavy to reread so this is our check poing

# creating unique id for videos
df['vid_id'] = df.groupby(['original_name']).ngroup()
df.head()

,date_,user_id_maped,program_name,duration_seconds,program_class,season,episode,program_desc,program_genre,series_title,hd,original_name,vid_id
0,2017-05-27,26138,100 treets,40,MOVIE,0,0,Drama Movie100 Streets,Drama,0,0,100 treets,2
1,2017-05-21,7946,Moana,17,MOVIE,0,0,Animation MovieMoana (HD),Animation,0,1,Moana,924
2,2017-08-10,7418,The Mermaid Princess,8,MOVIE,0,0,Animation MovieThe Mermaid Princess (HD),Animation,0,1,The Mermaid Princess,1524
3,2017-07-26,19307,The Mermaid Princess,76,MOVIE,0,0,Animation MovieThe Mermaid Princess (HD),Animation,0,1,The Mermaid Princess,1524
4,2017-07-07,15860,Churchill,87,MOVIE,0,0,Biography MovieChurchill (HD),Biography,0,1,Churchill,317


In [4]:
df['original_name'] = [" ".join(x.split()) for x in df['original_name']]

# get the names of TVs and movies
movie_orig = list(set(df[df['program_class'] == 'MOVIE']['original_name']))
tv_orig = list(set(df[df['program_class'] != 'MOVIE']['original_name']))



In [5]:
#number of movies
format(len(movie_orig))

'1525'

In [6]:
#number of shows
format(len(tv_orig))

'276'

In [7]:
def get_tmdb_id(names, types):

    item_api_id = []
    item_name_not_found = []
    item_dic = []
    for name in names:
        search  = types.search(name) # search api for names
        
        if len(search) != 0: # if nothing, meant we do not have the name in tmdb
            search = search[0]
            item_dic.append(search) # add found items in list
            item_api_id.append(search.id)
        else:
            item_name_not_found.append(name)
    # return 3 
    return item_api_id, item_name_not_found, item_dic

In [8]:
movie_api_id, movie_name_not_found, movie_dic = get_tmdb_id(movie_orig, movie)
tv_api_id, tv_name_not_found, tv_dic = get_tmdb_id(tv_orig, tv)

In [9]:
print(movie_dic[0])


{'adult': False, 'backdrop_path': '/qZEWyjSGX7xUpzLLJvF1XVtNmkh.jpg', 'genre_ids': [16, 10751, 12], 'id': 459037, 'original_language': 'zh', 'original_title': '美人鱼之海盗来袭', 'overview': "The adventures of a kind-hearted mermaid who saves a captain's daughter from the stormy ocean, despite the Sea-God's warning not to intervene in the affairs of mortals. However when the captain needs her help again, the mermaid princess must choose between following the laws of the ocean and saving her new human friends from a band of pirates and a dangerous island giant.", 'popularity': 2.962, 'poster_path': '/74eqnvi1GiveYcrO4WLopBPLoJZ.jpg', 'release_date': '2015-07-31', 'title': 'The Mermaid Princess', 'video': False, 'vote_average': 3.8, 'vote_count': 3}


In [10]:

# get all the genres ids list from tmdb api bcz all i get is genre ID
genres_tv = genre.tv_list()
genres_movie = genre.movie_list()
genres_tv.extend(genres_movie)
all_genres = genres_tv



In [11]:
#numer Shows in the dataset 
format(len(tv_orig))

'276'

In [12]:
#number of movies in the dataset 
format(len(movie_orig))

'1525'

In [13]:
#number of shows not found in TMDB

format(len(tv_name_not_found))

'141'

In [14]:
#number of films not found in TMDB

format(len(movie_name_not_found))


'192'

____________________________

# iwant to save the dictionary of movies and tv shows in to csv

In [15]:
def exp_dic(dic, filename):
    import csv
    csv_columns = list(dic[0].keys())
    dict_data = dic
    csv_file = "{}.csv".format(filename)
    try:
        with open(csv_file, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            writer.writeheader()
            for data in dict_data:
                writer.writerow(data)
    except IOError:
        print("error")

In [16]:
exp_dic(tv_dic, 'tv_df')
exp_dic(movie_dic, 'movie_df')
exp_dic(all_genres, 'genre_df')

In [17]:
# loading the dataframes
movie_df  = pd.read_csv('movie_df.csv')
tv_df     = pd.read_csv('tv_df.csv')
genre_df  = pd.read_csv('genre_df.csv')
genre_df  = genre_df.drop_duplicates()

# adding program type 
movie_df['program_type'] = 'Movie'
tv_df['program_type'] = 'TV Show'

In [18]:
def change_string_list(df, column):
   
    print('Before: {}, type: {}'.format(df[column][1], type(df[column][1])))
    
    for i in range(len(df[column])):
        df[column][i] = json.loads(df[column][i])
    
    print('After: {}, type: {}'.format(df[column][1], type(df[column][1])))

In [19]:
change_string_list(movie_df, 'genre_ids')
change_string_list(tv_df, 'genre_ids')


Before: [10751, 16, 12], type: <class 'str'>
After: [10751, 16, 12], type: <class 'list'>
Before: [10765], type: <class 'str'>
After: [10765], type: <class 'list'>


In [20]:
def Id_into_genre(df):
    print('Before: {}'.format(df['genre_ids'][1]))

    for i in range(len(df)):
        lists = df['genre_ids'][i]
        names = list(genre_df[genre_df['id'].isin(lists)]['name'])
        df.loc[:,'genre_ids'][i] = names
          
    print('After: {}'.format(df['genre_ids'][1]))

In [21]:
Id_into_genre(tv_df)
Id_into_genre(movie_df)

Before: [10765]
After: ['Sci-Fi & Fantasy']
Before: [10751, 16, 12]
After: ['Animation', 'Family', 'Adventure']


In [22]:
tv_df.head()


,backdrop_path,first_air_date,genre_ids,id,name,origin_country,original_language,original_name,overview,popularity,poster_path,vote_average,vote_count,program_type
0,/gZzIQxo2ukFSdZGtlttA0wI6kax.jpg,2011-06-05,"[Drama, Sci-Fi & Fantasy]",34524,Teen Wolf,['US'],en,Teen Wolf,"Scott McCall, a high school student living in ...",216.289,/vY2vfAskJTiWsQSv6bdbNCQhPLm.jpg,8.6,3337,TV Show
1,/uPoPcVwRbHG34p0VwzGQ4aXb8n2.jpg,2015-09-24,[Sci-Fi & Fantasy],60858,Heroes Reborn,['US'],en,Heroes Reborn,The supernatural phenomenon begins again with ...,17.765,/bHSkrlwjJgHq8Aa6AdDJvWuSI2X.jpg,6.9,403,TV Show
2,/jgb9xICHFX8wTGQ75js4uNbttQs.jpg,2015-10-26,"[Action & Adventure, Drama, Sci-Fi & Fantasy]",62688,Supergirl,['US'],en,Supergirl,"Twenty-four-year-old Kara Zor-El, who was take...",117.697,/zsaiq8ZclPuneuN7loLEbsh1ANJ.jpg,7.3,3703,TV Show
3,/2Ne8sYfnnRVl9BaghudN8bSkMnW.jpg,1997-03-31,"[Family, Kids]",1682,Teletubbies,['GB'],en,Teletubbies,"Pre-school fun, fantasy and education with col...",42.374,/orb9uGxgXs1Lo69SWwWeS5MxA2l.jpg,4.0,102,TV Show
4,/zkwO726aAsVVoDqjoH669HO0VO2.jpg,2011-11-06,"[Action & Adventure, Drama, Western]",1401,Hell on Wheels,['US'],en,Hell on Wheels,"The epic story of post-Civil War America, focu...",41.659,/w24HTBQG6bYMlWUKSrv1uceVfXF.jpg,7.6,329,TV Show


In [72]:
movie_df.head()

,id,original_title,release_date,program_type,genre_ids,overview,popularity,vote_average,vote_count
0,459037,美人鱼之海盗来袭,2015-07-31,Movie,"[Animation, Family, Adventure]",The adventures of a kind-hearted mermaid who s...,2.962,3.8,3
1,9325,The Jungle Book,1967-10-18,Movie,"[Animation, Family, Adventure]",The boy Mowgli makes his way to the man-villag...,89.700,7.3,5386
2,14560,Paul Blart: Mall Cop,2009-01-15,Movie,"[Comedy, Family, Action, Adventure]",Mild-mannered Paul Blart has always had huge d...,41.778,5.4,2094
3,180299,The Raid 2: Berandal,2014-03-27,Movie,"[Crime, Action, Thriller]",After fighting his way through an apartment bu...,46.277,7.6,1912
4,10426,Friday After Next,2002-11-22,Movie,[Comedy],Craig and his cousin Day Day have finally move...,16.382,6.2,316


In [24]:
# change the names of the columns so that it is the same as movie df 
tv_df.columns = ['backdrop_path', 'release_date', 'genre_ids', 'id', 'name',
       'origin_country', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'vote_average', 'vote_count',
       'program_type']

column_needed = ['id', 'original_title','release_date','program_type', 
                 'genre_ids', 'overview', 'popularity', 'vote_average', 'vote_count']

movie_df = movie_df[column_needed]
tv_df = tv_df[column_needed]

dfmerged = [movie_df, tv_df]
df  = pd.concat(dfmerged).reset_index()
df  = df.drop('index', axis=1)

df.columns = ['movie_id', 'original_name','release_date',
              'program_type', 'genres', 'overview', 
              'popularity', 'vote_average', 'vote_count']

# removing  from genres
df['genres'] = [", ".join(x) for x in df['genres']]
# the release date
df['release_date'] = [str(x)[:4] for x in df['release_date']]

df.head(3)


,movie_id,original_name,release_date,program_type,genres,overview,popularity,vote_average,vote_count
0,459037,美人鱼之海盗来袭,2015,Movie,"Animation, Family, Adventure",The adventures of a kind-hearted mermaid who s...,2.962,3.8,3
1,9325,The Jungle Book,1967,Movie,"Animation, Family, Adventure",The boy Mowgli makes his way to the man-villag...,89.700,7.3,5386
2,14560,Paul Blart: Mall Cop,2009,Movie,"Comedy, Family, Action, Adventure",Mild-mannered Paul Blart has always had huge d...,41.778,5.4,2094


fetching what i found in the API and matching it with the main dataset

In [25]:
#  only stc df to match found movies full information
new = list(df['original_name'].values)
# made a new  dataframe
stclast = stc[stc['original_name'].isin(new)]
stc_working = stc[~stc['original_name'].isin(new)]

In [26]:
# concat columns for better info about the content
complete_df = pd.merge(stclast, df, how='left', indicator=True)
complete_df.head()

,date_,user_id_maped,program_name,duration_seconds,program_class,season,episode,program_desc,program_genre,series_title,...,original_name,movie_id,release_date,program_type,genres,overview,popularity,vote_average,vote_count,_merge
0,2017-05-21,7946,Moana,17,MOVIE,0,0,Animation MovieMoana (HD),Animation,0,...,Moana,277834,2016,Movie,"Animation, Comedy, Family, Adventure","In Ancient Polynesia, when a terrible curse in...",47.750,7.6,10631,both
1,2017-07-07,15860,Churchill,87,MOVIE,0,0,Biography MovieChurchill (HD),Biography,0,...,Churchill,399790,2017,Movie,"Drama, History",A ticking-clock thriller following Winston Chu...,15.256,6.1,260,both
2,2018-03-29,6358,Coco,14,MOVIE,0,0,Animation MovieCoco (HD),Animation,0,...,Coco,354912,2017,Movie,"Animation, Comedy, Family, Adventure, Fantasy,...",Despite his family’s baffling generations-old ...,234.074,8.2,16037,both
3,2018-01-27,11660,Kidnap,85,MOVIE,0,0,Action MovieKidnap (HD),Action,0,...,Kidnap,293768,2017,Movie,"Drama, Thriller",A mother (in her Minivan) stops at nothing to ...,35.231,6.2,1130,both
4,2017-03-30,5155,The Accountant,42,MOVIE,0,0,Action MovieThe Accountant (HD),Action,0,...,The Accountant,302946,2016,Movie,"Crime, Drama, Thriller",As a math savant uncooks the books for a new c...,35.596,7.0,5158,both


In [27]:
def processing_dataframe(df):

    df.columns = ['stream_date', 'user_id', 'program_name', 'duration_seconds',
       'program_class', 'season', 'episode', 'program_desc', 'program_genre',
       'series_title', 'hd', 'original_name', 'program_id', 'release_date',
        'program_type', 'genres', 'overview', 'popularity', 'vote_average',
        'vote_count', 'merge']
    
    df['total_views'] = df.groupby('original_name', as_index=False)['user_id'].transform(lambda x: x.count())
    # feature: for content, ['overview', 'genres', 'release_date']
    df['genres'] = [x.replace(',', '') for x in df['genres']]

    df['content'] = df['genres']+  df['release_date'] + df['overview']
    
    df = df.drop_duplicates(subset = 'original_name').reset_index()
    df = df.drop('index', axis=1)

    df = df[['original_name', 'release_date', 'genres', 'overview','total_views', 'popularity', 'vote_average', 'vote_count', 'content']]
    return df

In [28]:
recomendation_df = processing_dataframe(complete_df)


In [29]:
recomendation_df.sample(5)

,original_name,release_date,genres,overview,total_views,popularity,vote_average,vote_count,content
938,The Reaping,2007,Horror,"Katherine Morrissey, a former Christian missio...",34,16.984,5.7,669,"Horror2007Katherine Morrissey, a former Christ..."
836,Four Brothers,2005,Crime Action,Four adopted brothers return to their Detroit ...,160,30.768,6.7,1711,Crime Action2005Four adopted brothers return t...
842,Shooter,2007,Crime Drama Mystery Action Thriller,A marksman living in exile is coaxed back into...,392,50.624,7.1,3931,Crime Drama Mystery Action Thriller2007A marks...
41,Premium Rush,2012,Crime Action Thriller,"In Manhattan, a bike messenger picks up an env...",1456,17.953,6.3,1798,"Crime Action Thriller2012In Manhattan, a bike ..."
569,Remember the Titans,2000,Drama,After leading his football team to 15 winning ...,150,22.965,7.6,2174,Drama2000After leading his football team to 15...


In [30]:
recomendation_df.to_csv("compstc.csv")


In [31]:
# feature extracting




In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline


In [33]:
def s_pipe_df(text, df=recomendation_df):
    content = np.array(df[text])
    
    pipe = Pipeline([('count', TfidfVectorizer(stop_words='english')),
                     ('tfid', TfidfTransformer())]).fit(content)

    mat_pip = pipe['count'].fit_transform(content)

    # converting it back to see 
    df = pd.DataFrame(cosine_similarity(mat_pip, mat_pip))
    
    return df


In [35]:
cos_sim = s_pipe_df('content')
cos_sim.head()

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,1.000000,0.000000,0.047532,0.000000,0.000000,0.0,0.007382,0.0,0.007150,0.004062,...,0.000000,0.000000,0.000000,0.014656,0.004940,0.017402,0.018127,0.042548,0.000000,0.000000
1,0.000000,1.000000,0.037758,0.015106,0.009211,0.0,0.000000,0.0,0.000000,0.000000,...,0.003761,0.010772,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008338,0.007293
2,0.047532,0.037758,1.000000,0.000000,0.000000,0.0,0.004734,0.0,0.012111,0.044592,...,0.005980,0.000000,0.004482,0.048325,0.003168,0.054549,0.026676,0.025141,0.012026,0.000000
3,0.000000,0.015106,0.000000,1.000000,0.011830,0.0,0.000000,0.0,0.000000,0.000000,...,0.022336,0.013836,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010709,0.045178
4,0.000000,0.009211,0.000000,0.011830,1.000000,0.0,0.028151,0.0,0.027268,0.008197,...,0.002945,0.008436,0.000000,0.000000,0.008872,0.000000,0.000000,0.000000,0.006530,0.005711


In [56]:
#Working On A Function To Where I give A name and It gives me 5 Similar movies
def get_recs(title, top, mat_df=cos_sim):
    # Get the index
    indices = pd.Series(recomendation_df.index, index=recomendation_df['original_name']).drop_duplicates()

    idx = indices[title]

    # Get the similarities
    sim_ids = list(enumerate(cos_sim[idx]))

    # Sort  based on  similarity 
    sim_ids = sorted(sim_ids, key=lambda x: x[1], reverse=True)
    # Get the scores of similar movies
    sim_ids = sim_ids[1:top+1]

    # Get indices
    item_indices = [i[0] for i in sim_ids]
    
    item_rec = recomendation_df['original_name'].iloc[item_indices]

    # Return 5 similar items
    return item_rec


get_recs('The Muppet Movie',5)

184    The Princess and the Frog
367          Muppets Most Wanted
337                  The Muppets
771                         Bolt
567                Dancing Ninja
Name: original_name, dtype: object

In [48]:
recomendation_df.sample(5)

,original_name,release_date,genres,overview,total_views,popularity,vote_average,vote_count,content
660,The Muppet Movie,1979,Comedy Family Adventure Music,A Hollywood agent persuades Kermit the Frog to...,182,12.771,7.1,455,Comedy Family Adventure Music1979A Hollywood a...
168,The November Man,2014,Crime Action Thriller,An ex-CIA operative is brought back in on a ve...,4829,25.193,6.1,1274,Crime Action Thriller2014An ex-CIA operative i...
819,Star Trek II: The Wrath of Khan,1982,Action Adventure Science Fiction Thriller,It is the 23rd century. The Federation Starshi...,72,17.408,7.5,1536,Action Adventure Science Fiction Thriller1982I...
26,No Reservations,2007,Comedy Drama Romance,Master chef Kate Armstrong runs her life and h...,381,13.328,6.5,1039,Comedy Drama Romance2007Master chef Kate Armst...
657,Deep Impact,1998,Drama Action Science Fiction,A seven-mile-wide space rock is hurtling towar...,316,31.450,6.2,2454,Drama Action Science Fiction1998A seven-mile-w...


In [77]:
get_recs('Star Trek II: The Wrath of Khan',5)

523      Star Trek: The Motion Picture
838    Star Trek V: The Final Frontier
540            Star Trek Into Darkness
888            Star Trek: Insurrection
396                            Gravity
Name: original_name, dtype: object

In [151]:
get_recs("Batman: Assault on Arkham",5)

712              The Brave One
578        Wrong Turn at Tahoe
87                    Flypaper
3                       Kidnap
39     Batman and Harley Quinn
Name: original_name, dtype: object

In [152]:
get_recs('The Muppet Movie',5)

184    The Princess and the Frog
367          Muppets Most Wanted
337                  The Muppets
771                         Bolt
567                Dancing Ninja
Name: original_name, dtype: object